In [2]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC

In [5]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\MODIS_BA"   # I should change this so I can work from github?
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [6]:
# select first MODIS file
MODISFiles = glob.glob('MCD64A1**.hdf') 
print(MODISFiles[0])

MCD64A1.A2002001.h19v07.006.2017012203845.hdf


In [7]:
# Just makes clear MODIS File naming convention and changes it from Julian date (YYYYDDD)
# Not actually changing the file name
productId = MODISFiles[0].split('.')[0]                                         # First: product name
yeardoy = MODISFiles[0].split(productId + '.A')[1].split('.')[0]                # Julian date
date = dt.datetime.strptime(yeardoy, '%Y%j').strftime('%d/%m/%Y')               # Convert YYYYDDD to DD/MM/YYYY
tiles_id = MODISFiles[0].split(yeardoy + '.')[1].split('.006')[0]               # Second: tile id
unknown_format = MODISFiles[0].split(tiles_id +'.006.')[1].split('.hdf')[0]     # Third: date (DD/MM/YYYY)


print('Product Name: {}\nLayer ID: {}\nDate of Observation: {}\n(Not sure what this represents: {})'.format(productId, tiles_id, date, unknown_format))

Product Name: MCD64A1
Layer ID: h19v07
Date of Observation: 01/01/2002
(Not sure what this represents: 2017012203845)


In [8]:
# View dataset metadata at the highest level of file - Not sure if this actually tells me anything useful
with rio.open(MODISFiles[0]) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

hdf4_meta

<open DatasetReader name='MCD64A1.A2002001.h19v07.006.2017012203845.hdf' mode='r'>


C:\Users\samgr\miniconda3\envs\GDAL_test\lib\site-packages\rasterio\__init__.py:218: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [9]:
# Print all of the subdatasets in the file - I think NDVI and EVI are the ones I'm interested in
with rio.open(MODISFiles[0]) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

HDF4_EOS:EOS_GRID:MCD64A1.A2002001.h19v07.006.2017012203845.hdf:MOD_Grid_Monthly_500m_DB_BA:Burn Date
HDF4_EOS:EOS_GRID:MCD64A1.A2002001.h19v07.006.2017012203845.hdf:MOD_Grid_Monthly_500m_DB_BA:Burn Date Uncertainty
HDF4_EOS:EOS_GRID:MCD64A1.A2002001.h19v07.006.2017012203845.hdf:MOD_Grid_Monthly_500m_DB_BA:QA
HDF4_EOS:EOS_GRID:MCD64A1.A2002001.h19v07.006.2017012203845.hdf:MOD_Grid_Monthly_500m_DB_BA:First Day
HDF4_EOS:EOS_GRID:MCD64A1.A2002001.h19v07.006.2017012203845.hdf:MOD_Grid_Monthly_500m_DB_BA:Last Day


In [10]:
file_name = MODISFiles[0]
file = SD(file_name, SDC.READ)  # SDC means scientific dataset  

print( file.info() )  # means there is 11 scientific datasets, although im not sure what the 6 signifies

(5, 17)


In [11]:
# print datasets in file
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print(idx,sds)

0 Burn Date
1 Burn Date Uncertainty
2 QA
3 First Day
4 Last Day


In [21]:
# get data for Burn Date - can change this to get data for EVI or other simply by changing the index number
sds_obj = file.select(0) # select sds

data = sds_obj.get() # get sds data
print(data)
# print attributes to see scale factor and add_offset
import pprint
pprint.pprint( sds_obj.attributes() )

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'_FillValue': -1,
 'long_name': 'ordinal day of burn',
 'valid_range': [0, 366],
 'water': -2}


In [33]:
print(data.flags)
print(data.shape, "(A total of", 2400*2400, "pixels)")

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

(2400, 2400) (A total of 5760000 pixels)


In [46]:
# this shows where fire burns for certain day 
result = np.where(data != 0)
print('Tuple of arrays returned : ', result)

# somehow get 2 different arrays?

Tuple of arrays returned :  (array([ 288,  321,  321, ..., 2399, 2399, 2399], dtype=int64), array([ 931,  873,  874, ..., 2381, 2382, 2383], dtype=int64))
